In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import plotly.express as px
#import seaborn as sns
import requests
from io import StringIO

In [ ]:
# Entire_home_apt.csv
# id_neighbordhood_date.csv

#dwn_url='https://figshare.com/articles/dataset/Entire_home_apt_csv/13259795'
#url = requests.get(dwn_url).text
#csv_raw = StringIO(url)

#url

In [ ]:
df=pd.read_csv('../Final/Berlin/merge_data_files/Entire_home_apt.csv',low_memory=False,index_col=0) 
df

In [ ]:

df.loc[df.date=='2020-05-14'].host_id.nunique()

In [ ]:
#find the top 20 districts

top10=df.loc[df.date=='2020-05-14']['neighbourhood_cleansed'].value_counts()[:10].index.tolist()
top10

In [ ]:
df2=df.groupby(['neighbourhood_cleansed','date'],as_index=False).count()[['neighbourhood_cleansed','date','id']]

In [ ]:
#df2=df2.sort_values(by=['id'], ascending=False)
df2.date = pd.to_datetime(df2.date, format='%Y-%m-%d')

In [ ]:
toplot=df2.loc[df2['neighbourhood_cleansed'].isin(top15)]
toplot

In [ ]:
tp=toplot.pivot(index='date', columns='neighbourhood_cleansed', values='id').reset_index().rename_axis(None, axis=1)

In [ ]:



import plotly.express as px
toplot = df2.loc[df2['neighbourhood_cleansed'].isin(top10)]
fig = px.line(toplot, x="date", y="id", color='neighbourhood_cleansed')
fig.show()

In [ ]:
o = px.data.gapminder().query("continent=='Oceania'")
o

In [ ]:
#find the top 20 hots

top20=df.loc[df.date=='2020-05-14']['host_id'].value_counts()[:30].index.tolist()
top20

In [ ]:
df3=df.groupby(['host_id','date'],as_index=False).count()[['host_id','date','id']]

In [ ]:
df3=df3.sort_values(by=['id'], ascending=False)
df3.date = pd.to_datetime(df3.date, format='%Y-%m-%d')

In [ ]:
toplot=df3.loc[df3['host_id'].isin(top20)]

In [ ]:
toplot

In [ ]:
toplot=df3.loc[df3['host_id'].isin(top20)]

plt.figure(figsize=(15, 10)) 



sns.lineplot(data=toplot, x="date", y="id", hue='host_id', markers=True, dashes=False)

In [ ]:
toplot=df3.copy()

bins = [0,1,2,4,8,16,260]

toplot['binned'] = pd.cut(toplot['id'], bins)



toplot

In [ ]:
df['binned_calc']=pd.cut(df['calculated_host_listings_count'], bins)

In [ ]:
df['binned_calc']=df['binned_calc'].astype(str)

In [ ]:
pie_5=df.groupby(['binned_calc','date'],as_index=False).count()[['binned_calc','date','id']]

In [ ]:
pie_5.date

In [ ]:
pie_5.loc[pie_5.date=='2016-05-03']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['2016','2020'])
fig.add_trace(go.Pie(labels=pie_5.loc[pie_5.date=='2016-05-03'].binned_calc, 
                     values=pie_5.loc[pie_5.date=='2016-05-03'].id, scalegroup='one',
                     name="2016"), 1, 1)
fig.add_trace(go.Pie(labels=pie_5.loc[pie_5.date=='2020-05-14'].binned_calc, 
                     values=pie_5.loc[pie_5.date=='2020-05-14'].id, scalegroup='one',
                     name="2020"), 1, 2)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0), height=600
)

fig.update_layout(title_text='Multiple flat host evolution')
fig.show()
fig.write_html("../Final/Pie_chart_5years.html")

In [ ]:
# count the multiple host listings
plot=toplot.groupby(['binned','date'],as_index=False).count()

In [ ]:
fig = px.bar(plot, x='date', y='id', labels = {"value": "count", "variable": "medal"},
             color='binned', 
            title='Number of hosts with multiple listings_only_apts')
fig.show()
fig.write_html("../Final/multipleL_host_5years.html")

# plot the multiple flat hosts over time on map

In [ ]:
#toplot.loc[
toplot['binned']=toplot['binned'].astype(str)


In [ ]:
data2=toplot.loc[toplot['binned'].isin(['(2, 4]','(4, 8]','(8, 16]','(16, 260]'])][['date','host_id','binned']]

In [ ]:
df.dtypes

In [ ]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')

In [ ]:
data2.dtypes

In [ ]:
data=pd.merge(df, data2,  how='left',  on=['host_id','date'])


In [ ]:
# clean data for ploting
#df=data.loc[data.binned.notnull()]
df.date=df.date.apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
import plotly.figure_factory as ff

# it needs a token for access mapbox
px.set_mapbox_access_token('pk.eyJ1Ijoiam9obnppbnoiLCJhIjoiY2tmbWthazZ6MDNueDJxb2ZyZ2M3czU0dyJ9.Bl3T4kl14xan7glGxid_Rw')


lat = df['latitude']
lon = df['longitude']
frame = df['date']

fig = ff.create_hexbin_mapbox(
    lat=lat, lon=lon, nx_hexagon=50, animation_frame=frame,
    color_continuous_scale="Viridis", labels={"color": "Airbnb listings > 50", "frame": "Period"},
    opacity=0.5, min_count=60, height=500, zoom=10)
    #show_original_data=True, original_data_marker=dict(opacity=0.4, size=3, color="blue"))
fig.update_layout(title={'text': "density spots"})
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

# write to an html file
fig.write_html("../Final/hives_for_density2.html")